# 7.5 PDE-Constrained Shape Optimization (fully automated)

We want to solve the PDE-constrained shape optimization problem
$$
            \underset{\Omega\subset \mathsf{D}}{\mbox{min}} \; J(u) := \int_\Omega |u-u_d|^q \; dx, \quad q\ge 2
$$
subject to that $(\Omega,u)$ satisfy
$$
           \int_\Omega \nabla u \cdot \nabla v \; dx = \int_\Omega f v \; dx \; \quad \text{ for all } v \in H_0^1(\Omega),
$$
where $\Omega \subset \mathbb R^2$ for given $u_d, f \in C^1(\mathbb R^2)$.

Again, we want to compute the shape derivative by differentiation of a suitably defined perturbed Lagrangian using automated differentiation. Here this is accounted for automatically by NGSolve. For details we refer to 

P. Gangl, K. Sturm, M. Neunteufel, J. Schöberl.
Fully and Semi-Automated Shape Differentiation in NGSolve,
Struct. Multidisc. Optim., 63, pp.1579-1607, 2021.

In [22]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import SplineGeometry
from ngsolve.solvers import *

In [23]:
geo = SplineGeometry()
geo.AddCircle(c=(0.5,0.5), r=0.5, bc = 'circle')
mesh = Mesh(geo.GenerateMesh(maxh = 0.08))
Draw(mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 1, 'order3d': 1, 'draw_vol': None…

BaseWebGuiScene

In [24]:
#given data of our problem (chosen such that \Omega^* = [0,1]^2 is the optimal shape)
f = CoefficientFunction(-2*y*(1-y)+2*x*(1-x))
ud = x*(1-x)*y*(1-y)

grad_f = CoefficientFunction( (f.Diff(x), f.Diff(y) ) )
grad_ud = CoefficientFunction( (ud.Diff(x), ud.Diff(y) ) )

In [25]:
fes = H1(mesh, order=2, dirichlet=".*")
gfu = GridFunction(fes)
scene_u = Draw (gfu, mesh, "state")

gfp = GridFunction(fes)
scene_p = Draw (gfp, mesh, "adjoint")

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

Note that (for linear problems) the operator on the left hand side of the adjoint equation is just the transpose of the state operator.

### Automatic Shape Differentiation

The formula for the shape derivative was derived as the partial derivative of the perturbed Lagrangian (brought back to the original domain):
$$
    d\mathcal J(\Omega; X) = \frac{\partial}{\partial t} \left. \left( \int_\Omega |u - u_d^t|^q  \,\mbox{det}(F_t) \mbox{d} x 
         +  \int_{\Omega} (F_t^{-\top}\nabla u) \cdot (F_t^{-\top} \nabla p)\,  \mbox{det}(F_t) \, dx - \int_{\Omega}f^t p  \,\mbox{det}(F_t) \,dx \right) \right \rvert_{t=0} 
$$
where 
<ul>
    <li>   $T_t(x)=x+tX(x)=y$ 
    <li> $F_t = DT_t = I+t DX$
    <li>  $u_d^t = u_d \circ T_t$
    <li> $f^t = f \circ T_t$
</ul>

The integrand depends on the parameter $t$ only via $F_t$ and $T_t$. We define the Lagrangian in this form, involving a parameter t that has the value 0.

### In order to define the perturbed Lagrangian for a given problem, one needs to know transformation rules of differential operators, e.g., $$(\nabla u)\circ T_t = (\partial T_t)^{-T} \nabla (u \circ T_t)$$ for $u \in H^1$.
### Since these transformation rules are known by NGSolve for all implemented differential operators, also the step of defining the perturbed Lagrangian can be automated.

In [5]:
VEC = H1(mesh, order=2, dim=2)
PHI, X = VEC.TnT()

def EquationFA(u,v):
    return ( grad(u)*grad(v)-f*v)*dx

q=4
def CostAutoFA(u): 
    return (u-ud)**q*dx

def CostAuto2(u): 
    return CostAutoFA(u)

LagrangianFA = CostAutoFA(gfu) + EquationFA(gfu,gfp)

### State equation

Equation can also be used to define the bilinear form. The following defines left and right hand side of the PDE in a "BilinearForm":

In [6]:
u, v = fes.TnT()

aAuto = BilinearForm(fes, symmetric=True)
aAuto += EquationFA(u,v)

Now the PDE can be conveniently solved by calling Newton's method (which terminates after one iteration since the PDE is linear)

In [7]:
gfu.vec[:]=0
Newton(aAuto,gfu,freedofs=fes.FreeDofs())
scene_u.Redraw()

Newton iteration  0
err =  0.1304243597310235
Newton iteration  1
err =  1.2322859411137204e-16


### Adjoint equation

We set up the adjoint equation
$$
    \mbox{Find } p \in H_0^1(\Omega): \int_\Omega \nabla w \cdot \nabla p \, \mbox dx = - \partial_u J(u)(w) \quad \text{ for all } w \in H_0^1(\Omega)
$$
where $u$ is the solution to the state equation. For $J(u) = \int_\Omega |u-u_d|^2 \mbox dx$, we get
$$
    \partial_u J(u)(w) = 2 \int_\Omega (u-u_d)w \,\mbox dx.
$$
However, we can also use the Diff(...) command:

In [8]:
p, w = fes.TnT()

fadjoint = LinearForm(fes)
fadjoint += -1*(CostAutoFA(gfu)).Diff(gfu,w)

In [9]:
def SolveAdjointEquation():
    rhs = gfp.vec.CreateVector()
    rhs.data = fadjoint.vec - aAuto.mat.T * gfp.vec
    update = gfp.vec.CreateVector()
    update.data = aAuto.mat.Inverse(fes.FreeDofs()).T * rhs
    gfp.vec.data += update

In [10]:
fadjoint.Assemble()
SolveAdjointEquation()
scene_p.Redraw()

### Automatic Shape Differentiation

Denoting the integrand by $G^{u,p}$, the shape derivative is given by
$$
\begin{array}{rl}
     d\mathcal J(\Omega; X) =& \left( \left. \frac{\partial G^{u,p}}{\partial t} + \frac{d  G^{u,p}}{dy} \cdot \frac{d T_t}{dt}\right)\right\rvert_{t=0} \\
     =& \left.  \frac{\partial G^{u,p}}{\partial t}\right\rvert_{t=0} + \frac{d  G^{u,p}}{dy} \cdot X
\end{array}
$$
The command .DiffShape(...) computes the shape derivative by automatically accounting for the corresponding transformations.

In [11]:
dJOmegaAuto = LinearForm(VEC)
dJOmegaAuto += LagrangianFA.DiffShape(X)

In [12]:
b = BilinearForm(VEC)
b += InnerProduct(grad(X),grad(PHI))*dx + InnerProduct(X,PHI)*dx

gfX = GridFunction(VEC)

In [13]:
def SolveDeformationEquationAuto():
    rhs = gfX.vec.CreateVector()
    rhs.data = dJOmegaAuto.vec - b.mat * gfX.vec
    update = gfX.vec.CreateVector()
    update.data = b.mat.Inverse(VEC.FreeDofs()) * rhs
    gfX.vec.data += update

In [14]:
b.Assemble()
dJOmegaAuto.Assemble()
SolveDeformationEquationAuto()
Draw(-gfX, mesh, "-gfX")

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

BaseWebGuiScene

In [15]:
# gfset denotes the deformation of the original domain and will be updated during the shape optimization
gfset = GridFunction(VEC)
gfset.Set((0,0))
mesh.SetDeformation(gfset)
sceneSet = Draw(gfset,mesh,"gfset")
SetVisualization (deformation=True)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

In [16]:
gfset.Set((0,0))
mesh.SetDeformation(gfset)
print('Cost at initial design', Integrate (CostAuto2(gfu), mesh))

scale = 0.5 / Norm(gfX.vec)
gfset.vec.data -= scale * gfX.vec
sceneSet.Redraw()

Cost at initial design 5.1522441308147555e-09


In [17]:
gfu.vec[:]=0
Newton(aAuto, gfu, fes.FreeDofs())
print('Cost at new design', Integrate (CostAuto2(gfu), mesh))

Newton iteration  0
err =  0.15395715607378452
Newton iteration  1
err =  1.1887533241005314e-16
Cost at new design 9.793297083297825e-10


Thus, the user has to enter the PDE (in its transformed form) only once.

Finally, let us again run the full algorithm:

In [18]:
scene_u = Draw(gfu)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

In [19]:
#reset to and solve for initial configuration
gfset.Set((0,0))
mesh.SetDeformation(gfset)
scene_u.Redraw()
gfu.vec[:]=0
Newton(aAuto, gfu, fes.FreeDofs())

LineSearch = False


iter_max = 600
Jold = Integrate(CostAuto2(gfu), mesh)
converged = False
for k in range(iter_max):
    print('cost at iteration', k, ': ', Jold)
    mesh.SetDeformation(gfset)
    scene_u.Redraw()
    
    gfu.vec[:]=0
    Newton(aAuto, gfu, fes.FreeDofs(), printing = False)
    
    fadjoint.Assemble()
    SolveAdjointEquation()
    
    b.Assemble()
    dJOmegaAuto.Assemble()
    SolveDeformationEquationAuto()

    scale = 0.01 / Norm(gfX.vec)
    gfsetOld = gfset
    gfset.vec.data -= scale * gfX.vec
    
    Jnew = Integrate(CostAuto2(gfu), mesh)
    
    if LineSearch:
        while Jnew > Jold and scale > 1e-12:
            # input('a')
            scale = scale / 2
            
            if scale <= 1e-12:
                converged = True
                break

            gfset.vec.data = gfsetOld.vec - scale * gfX.vec
            mesh.SetDeformation(gfset)
            
            gfu.vec[:]=0
            Newton(aAuto, gfu, fes.FreeDofs(), printing = False)
            Jnew = Integrate(CostAuto(gfu), mesh)
    
    if converged==True:
        break
    Jold = Jnew

    Redraw(blocking=True)

Newton iteration  0
err =  0.1304243597310235
Newton iteration  1
err =  1.2322859411137204e-16
cost at iteration 0 :  5.1522441308147555e-09
cost at iteration 1 :  4.942040315052276e-09
cost at iteration 2 :  4.585081411492747e-09
cost at iteration 3 :  4.2494409805570896e-09
cost at iteration 4 :  3.934230926246942e-09
cost at iteration 5 :  3.638576261991293e-09
cost at iteration 6 :  3.3616160503340736e-09
cost at iteration 7 :  3.1025043360794835e-09
cost at iteration 8 :  2.8604110705929728e-09
cost at iteration 9 :  2.634523024596651e-09
cost at iteration 10 :  2.424044686339463e-09
cost at iteration 11 :  2.2281991414270806e-09
cost at iteration 12 :  2.0462289298150803e-09
cost at iteration 13 :  1.8773968744302314e-09
cost at iteration 14 :  1.7209868744933654e-09
cost at iteration 15 :  1.576304654739389e-09
cost at iteration 16 :  1.4426784591830553e-09
cost at iteration 17 :  1.319459674612464e-09
cost at iteration 18 :  1.206023364263461e-09
cost at iteration 19 :  1.1017

cost at iteration 174 :  1.2752929053208592e-15
cost at iteration 175 :  1.2309245708375936e-15
cost at iteration 176 :  1.1848590098259628e-15
cost at iteration 177 :  1.1453633964661322e-15
cost at iteration 178 :  1.1038963070915974e-15
cost at iteration 179 :  1.0687409569141703e-15
cost at iteration 180 :  1.0313554716748845e-15
cost at iteration 181 :  1.0000465057814174e-15
cost at iteration 182 :  9.662703793913344e-16
cost at iteration 183 :  9.383577195440387e-16
cost at iteration 184 :  9.077667798624242e-16
cost at iteration 185 :  8.828478178193601e-16
cost at iteration 186 :  8.550658529494464e-16
cost at iteration 187 :  8.327848317733904e-16
cost at iteration 188 :  8.074802263079748e-16
cost at iteration 189 :  7.875247064930154e-16
cost at iteration 190 :  7.644060164448333e-16
cost at iteration 191 :  7.46502476794992e-16
cost at iteration 192 :  7.253142004038751e-16
cost at iteration 193 :  7.092237167742292e-16
cost at iteration 194 :  6.8974243402945e-16
cost at 

cost at iteration 348 :  1.7054692486014585e-16
cost at iteration 349 :  1.722626268899642e-16
cost at iteration 350 :  1.6951005028429798e-16
cost at iteration 351 :  1.7124361205259585e-16
cost at iteration 352 :  1.6849904577621667e-16
cost at iteration 353 :  1.7025008119906862e-16
cost at iteration 354 :  1.6751304018180047e-16
cost at iteration 355 :  1.692811800643725e-16
cost at iteration 356 :  1.6655119881189567e-16
cost at iteration 357 :  1.6833608995763938e-16
cost at iteration 358 :  1.656127216023338e-16
cost at iteration 359 :  1.6741402597793034e-16
cost at iteration 360 :  1.646968413828787e-16
cost at iteration 361 :  1.6651423533503423e-16
cost at iteration 362 :  1.6380282224770616e-16
cost at iteration 363 :  1.6563599576818257e-16
cost at iteration 364 :  1.629299580205637e-16
cost at iteration 365 :  1.6477861405615513e-16
cost at iteration 366 :  1.6207757080833774e-16
cost at iteration 367 :  1.6394142461269099e-16
cost at iteration 368 :  1.6124500963717873e-

cost at iteration 522 :  1.286009069219109e-16
cost at iteration 523 :  1.3126373320024225e-16
cost at iteration 524 :  1.283880350588888e-16
cost at iteration 525 :  1.3105921316532938e-16
cost at iteration 526 :  1.281778616391342e-16
cost at iteration 527 :  1.3085738084104615e-16
cost at iteration 528 :  1.2797033243787841e-16
cost at iteration 529 :  1.3065818262028437e-16
cost at iteration 530 :  1.2776539455694835e-16
cost at iteration 531 :  1.3046156620218906e-16
cost at iteration 532 :  1.2756299638686515e-16
cost at iteration 533 :  1.302674805548625e-16
cost at iteration 534 :  1.273630875701952e-16
cost at iteration 535 :  1.3007587587928173e-16
cost at iteration 536 :  1.2716561896614907e-16
cost at iteration 537 :  1.2988670357440773e-16
cost at iteration 538 :  1.2697054261627274e-16
cost at iteration 539 :  1.2969991620343557e-16
cost at iteration 540 :  1.2677781171130866e-16
cost at iteration 541 :  1.295154674611427e-16
cost at iteration 542 :  1.2658738055910305e-1